# Delay Analysis

### 1️⃣ Loading Flight Data for Delay Analysis

Purpose:
Import required libraries and load the processed flight dataset into memory as a Pandas DataFrame. This dataset serves as the base input for delay analysis, aggregation, and visualization.

In [2]:
import pandas as pd
import numpy as np
flights_df = pd.read_csv("../data/flights.csv")


### 2️⃣ Timestamp Standardization for Delay Analysis

Purpose:
Convert scheduled and actual flight timestamps into proper datetime format and derive a flight date field to support accurate delay calculation and time-based analysis.

In [3]:
flights_df["scheduled_time"] = pd.to_datetime(
    flights_df["scheduled_time"], errors="coerce"
)

flights_df["actual_time"] = pd.to_datetime(
    flights_df["actual_time"], errors="coerce"
)
flights_df["delay_date"] = flights_df["scheduled_time"].dt.date


### 3️⃣ Flight Delay Calculation and Normalization

Purpose:
Compute flight delays in minutes by comparing actual and scheduled timestamps and normalize the results by removing negative values caused by early arrivals or departures.

In [4]:
flights_df["delay_min"] = (
    (flights_df["actual_time"] - flights_df["scheduled_time"])
    .dt.total_seconds() / 60
)

# keep only positive delays
flights_df.loc[flights_df["delay_min"] < 0, "delay_min"] = 0


### 4️⃣ Delay and Cancellation Flag Creation

Purpose:
Create binary indicator columns to classify flights based on operational performance. These flags enable easy filtering, aggregation, and comparison of delayed and cancelled flights during delay analysis.

In [5]:
# delayed flight flag (>=15 min)
flights_df["is_delayed"] = flights_df["delay_min"] >= 15

# cancelled flight flag
flights_df["is_cancelled"] = flights_df["status"].isin(
    ["Cancelled", "Canceled"]
)


### 5️⃣ Airport-Level Daily Delay Aggregation

Purpose:
Aggregate flight-level delay data to produce daily, airport-level performance metrics. This enables comparison of operational efficiency across airports and identification of delay and cancellation patterns over time.

In [7]:
airport_delays_df = (
    flights_df
    .groupby(["origin_iata", "delay_date"])
    .agg(
        total_flights=("flight_number", "count"),
        delayed_flights=("is_delayed", "sum"),
        avg_delay_min=("delay_min", "mean"),
        median_delay_min=("delay_min", "median"),
        canceled_flights=("is_cancelled", "sum")
    )
    .reset_index()
)


### 6️⃣ Final Cleanup and Formatting of Airport Delay Metrics

Purpose:
Standardize column naming and format delay metrics into clean, presentation-ready integer values to improve readability and consistency for reporting, SQL storage, and dashboard visualization.

In [8]:
airport_delays_df = airport_delays_df.rename(
    columns={"origin_iata": "airport_iata"}
)

airport_delays_df["avg_delay_min"] = (
    airport_delays_df["avg_delay_min"]
    .round()
    .astype("Int64")
)

airport_delays_df["median_delay_min"] = (
    airport_delays_df["median_delay_min"]
    .round()
    .astype("Int64")
)


In [9]:
airport_delays_df.to_csv(
    "../data/airport_delays.csv",
    index=False
)


In [10]:
airport_delays_df

,airport_iata,delay_date,total_flights,delayed_flights,avg_delay_min,median_delay_min,canceled_flights
0,ABJ,2026-01-02,2,0,0,0,0
1,ABX,2026-01-02,1,0,0,0,0
2,ABZ,2026-01-02,7,1,6,0,0
3,ACC,2026-01-02,1,0,0,0,0
4,ADB,2026-01-02,1,0,0,0,0
...,...,...,...,...,...,...,...
469,YYC,2026-01-02,3,0,0,0,0
470,YYZ,2026-01-02,11,1,27,0,0
471,ZAG,2026-01-02,4,1,6,0,0
472,ZNZ,2026-01-02,2,1,18,18,0
